plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

Pseudocode:
    1. Import
    2. Load Parameters
    3. For Loop through seizures

In [117]:
#1. Import
import csv
import os
import pathlib
import numpy as np
import openpyxl
from scipy.io import loadmat


############################################################

#2. Load Parameters
opscea_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/' # set path

os.chdir(opscea_path)
params_xlsx = openpyxl.load_workbook('OPSCEAparams.xlsx')
params_sheet = params_xlsx.active
dimensions = params_sheet.dimensions
dim_1 = dimensions.split(':')[0] # top leftmost cell
dim_2 = dimensions.split(':')[1] # bottom rightmost cell
params_row_start = "" #string to hold digits of dim_2
params_row_end = "" #string to hold digits of dim_2
col_range_list = []
for char in dim_1:
    if char.isdigit():
        params_row_start += char
for char in dim_2:
    if char.isdigit():
        params_row_end += char

alphabet =[]
for i in range(65,82):
    alphabet.append(chr(i))
print(alphabet)
        
params_header = ':'.join([dim_1,alphabet[len(alphabet)-1]+params_row_start])

for letter in alphabet:
    # str_to_analyze1 = letter+params_row_start
    # str_to_analyze2 = letter+params_row_end
    col_range = ':'.join([letter+params_row_start,letter+params_row_end])
    col_range_list.append(col_range)
    
    # col_data = params_sheet[
    #                         ] #A1:A14
    
print(col_range_list[0])

# pt_col = params_sheet[col_range_list[0]]
# print(pt_col.value)

# for patient in col_range_list

# if params_sheet['A2:'] == pt_folder:

# VIDstart = params_sheet['D']
# name = params_sheet['D3']
# tag = params_sheet.cell(row = 1, column = 2)

# print(name.value)
# print(tag.value)


############################################################

#3. For loop through seizures
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'
os.chdir(avg_change_path)


pt_list = os.listdir(avg_change_path)

new_pt_list = []
anat = []

#delete folders that do not contain seizures
for pt_i in range(0,len(pt_list)):
    pt = pt_list[pt_i]
    if pt[0] != '.':
        new_pt_list.append(pt)

        

for pt_folder in new_pt_list:
    path1 = os.path.join(avg_change_path,pt_folder)
    # print(path1)
    os.chdir(path1)
    sz_list = os.listdir(path1)
        
    #collect seizure names by ignoring folders that do not contain seizures
    for sz_i in range(0,len(sz_list)):
        sz_name = sz_list[sz_i]
        if sz_name[0] != '.' and sz_name != 'Imaging':
            path2 = os.path.join(path1,sz_name)
            path3 = os.path.join(path1,'Imaging/elecs')

        #load symptom matrix (mat.csv) and time matrix (time_mat.csv) and electrode info over time matrix (sz.mat)
            os.chdir(path2)

            mat_file = '_'.join([sz_name,'mat.csv'])
            time_mat_file = '_'.join([sz_name,'time_mat.csv'])
            sz_mat_file = ''.join([sz_name,'.mat'])
        
            ## mat.csv
            with open(mat_file,'r') as sz_mat:
                mat_csv_list = list(csv.reader(sz_mat))
                
                #collect symptom abbreviations
                all_sx = mat_csv_list[0] 
                
            ## time_mat.csv
            with open(time_mat_file,'r') as sz_time_mat:
                time_mat_csv_list = list(csv.reader(sz_time_mat))

            ## sz.mat
            sz_mat = loadmat(sz_mat_file)                            
            
                #load fs, ppEEG, study
            fs = [[element for element in upperElement] for upperElement in sz_mat['fs']]
            ppEEG = [[element for element in upperElement] for upperElement in sz_mat['ppEEG']]
            # study = [[element for element in upperElement] for upperElement in sz_mat['study']]
           
        
            if 'fs' in locals() or 'fs' in globals():
                sfx = fs
                del fs
                
            if 'ppEEG' in locals() or 'ppEEG' in globals():
                d = ppEEG
                del ppEEG


            #load electrodes -- anatomy, eleclabels, elecmatrix
            os.chdir(path3)
            
            elecs_mat = loadmat('clinical_elecs_all.mat')            
        
                #load anatomy
            anatomy = [[element for element in upperElement] for upperElement in elecs_mat['anatomy']]
            # eleclabels = [[element for element in upperElement] for upperElement in elecs_mat['eleclabels']]
            # elecmatrix = [[element for element in upperElement] for upperElement in elecs_mat['elecmatrix']]
            
            
            # make sure variables are present
            if 'anatomy' in locals() or 'anatomy' in globals():
                #collect neuroanatomy by row
                for a_row in range(0,np.size(anatomy,0)):
                    if np.size(anatomy[a_row][3]) == 1:
                        anat.append(anatomy[a_row][3][0])
                del anatomy
            

        
############################################################

#4. Initialize variables

                
# neuroanat_list = ['frontalpole', # FRONTAL LOBE
# 'parstriangularis',
# 'parsopercularis',
# 'parsorbitalis',
# 'rostralmiddlefrontal',
# 'caudalmiddlefrontal',
# 'lateralorbitofrontal',
# 'superiorfrontal',
# 'medialorbitofrontal',
# 'precentral',
# 'postcentral',                  # PARIETAL LOBE
# 'inferiorparietal',   
# 'superiorparietal',
# 'supramarginal',
# 'temporalpole',                 # TEMPORAL LOBE
# 'middletemporal',
# 'superiortemporal',
# 'inferiortemporal',
# 'parahippocampal',               
# 'Right-Hippocampus',
# 'Left-Hippocampus',
# 'Right-Amygdala',
# 'Left-Amygdala',
# 'entorhinal',
# 'bankssts',
# 'fusiform',                     # OCCIPITAL LOBE                
# 'lingual',
# 'Right-Inf-Lat-Vent',           # OTHER
# 'Right-Cerebral-White-Matter',
# 'Left-Cerebral-White-Matter',
# 'Right-choroid-plexus',
# 'Right-Putamen',
# 'Right-VentralDC']


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q']
A1:A14
